In [2]:
import os
import pandas as pd
current_dir = os.getcwd()
print("initial Working directory",current_dir)
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
print("Final Working Dir",current_dir)

initial Working directory h:\VScode\March Group\March_Team_Project\jupyter_notebooks
You set a new current directory
Final Working Dir h:\VScode\March Group\March_Team_Project


In [3]:
# load pollution_us_2000_2016.csv.zip
pollution = pd.read_csv('source_data/pollution_us_2000_2016.csv.zip', compression='zip')
# Drop rows with null values
pollution = pollution.dropna()
print(pollution.head())
print(pollution.info())
print(pollution.describe())
# produce summary by city, county, state, counting the number of records, and the number of unique dates add count of unique sites
pollution['Date Local'] = pd.to_datetime(pollution['Date Local'])
pollution['year'] = pollution['Date Local'].dt.year
pollution_summary = pollution.groupby(['City', 'County', 'State', 'year']).agg({'Date Local': ['count', 'nunique']})
pollution_summary.columns = ['Record Count', 'Unique Dates']
print(pollution_summary)

    Unnamed: 0  State Code  County Code  Site Num  \
1            1           4           13      3002   
5            5           4           13      3002   
9            9           4           13      3002   
13          13           4           13      3002   
17          17           4           13      3002   

                                    Address    State    County     City  \
1   1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
5   1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
9   1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
13  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   
17  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa  Phoenix   

    Date Local          NO2 Units  ...          SO2 Units  SO2 Mean  \
1   2000-01-01  Parts per billion  ...  Parts per billion  3.000000   
5   2000-01-02  Parts per billion  ...  Parts per billion  1.958333   
9   2000-01-03  P

In [4]:
# load pollution_us_2000_2016.csv.zip
pollution = pd.read_csv('source_data/pollution_us_2000_2016.csv.zip', compression='zip')
# list of columns
print(pollution.columns)
# extract location based data
location_data_sunday = pollution[['City', 'County', 'State', 'Address', 'State Code', 'County Code', 'Site Num']].drop_duplicates()
location_data_sunday


Index(['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address',
       'State', 'County', 'City', 'Date Local', 'NO2 Units', 'NO2 Mean',
       'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 Units',
       'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units',
       'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
       'CO Units', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],
      dtype='object')


,City,County,State,Address,State Code,County Code,Site Num
0,Phoenix,Maricopa,Arizona,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,4,13,3002
1396,Scottsdale,Maricopa,Arizona,2857 N MILLER RD-S SCOTTSDALE STN,4,13,3003
2128,Tucson,Pima,Arizona,"1237 S. BEVERLY , TUCSON",4,19,1011
3516,Concord,Contra Costa,California,2956-A TREAT BOULEVARD,6,13,2
4968,Bethel Island,Contra Costa,California,5551 BETHEL ISLAND RD,6,13,1002
...,...,...,...,...,...,...,...
1548535,Not in a city,Rockingham,New Hampshire,150 Pillsbury Rd,33,15,18
1557887,Raleigh,Wake,North Carolina,3801 SPRING FOREST RD.,37,183,14
1568833,Not in a city,Adams,Pennsylvania,NARSTO SITE - ARENDTSVILLE,42,1,1
1594263,Riverton,Fremont,Wyoming,"90 GAS HILL ROAD, RIVERTON, WY",56,13,6001


In [5]:
import sys
import os

# Add parent directory to sys.path if needed
sys.path.append(os.path.abspath("utils"))
from geo_utils import get_geolocation_info
# create unified address column
location_data_sunday['address'] = location_data_sunday['Address'] + ', ' + location_data_sunday['City'] + ', ' + location_data_sunday['State']
# get geolocation data
location_data_sunday[['latitude', 'longitude', 'formatted_address']] = location_data_sunday['address'].apply(
    lambda x: pd.Series(get_geolocation_info(x))
)

location_data_sunday

INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocation_info
INFO:utils.simple_logger:Calling: get_geolocation_info
INFO:utils.simple_logger:Completed: get_geolocati

,City,County,State,Address,State Code,County Code,Site Num,address,latitude,longitude,formatted_address
0,Phoenix,Maricopa,Arizona,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,4,13,3002,"1645 E ROOSEVELT ST-CENTRAL PHOENIX STN, Phoen...",33.458426,-112.046574,"1645 E Roosevelt St, Phoenix, AZ 85006, USA"
1396,Scottsdale,Maricopa,Arizona,2857 N MILLER RD-S SCOTTSDALE STN,4,13,3003,"2857 N MILLER RD-S SCOTTSDALE STN, Scottsdale,...",33.479765,-111.917239,"2857 N Miller Rd, Scottsdale, AZ 85257, USA"
2128,Tucson,Pima,Arizona,"1237 S. BEVERLY , TUCSON",4,19,1011,"1237 S. BEVERLY , TUCSON, Tucson, Arizona",32.206231,-110.879379,"1237 S Beverly Ave, Tucson, AZ 85711, USA"
3516,Concord,Contra Costa,California,2956-A TREAT BOULEVARD,6,13,2,"2956-A TREAT BOULEVARD, Concord, California",37.936272,-122.025806,"2956 Treat Blvd a, Concord, CA 94518, USA"
4968,Bethel Island,Contra Costa,California,5551 BETHEL ISLAND RD,6,13,1002,"5551 BETHEL ISLAND RD, Bethel Island, California",38.006241,-121.643328,"5551 Bethel Island Rd, Oakley, CA 94561, USA"
...,...,...,...,...,...,...,...,...,...,...,...
1548535,Not in a city,Rockingham,New Hampshire,150 Pillsbury Rd,33,15,18,"150 Pillsbury Rd, Not in a city, New Hampshire",42.863481,-71.379678,"150 Pillsbury Rd, Londonderry, NH 03053, USA"
1557887,Raleigh,Wake,North Carolina,3801 SPRING FOREST RD.,37,183,14,"3801 SPRING FOREST RD., Raleigh, North Carolina",35.856738,-78.576428,"3801 Spring Forest Rd, Raleigh, NC 27616, USA"
1568833,Not in a city,Adams,Pennsylvania,NARSTO SITE - ARENDTSVILLE,42,1,1,"NARSTO SITE - ARENDTSVILLE, Not in a city, Pen...",39.923149,-77.298596,"Arendtsville, PA, USA"
1594263,Riverton,Fremont,Wyoming,"90 GAS HILL ROAD, RIVERTON, WY",56,13,6001,"90 GAS HILL ROAD, RIVERTON, WY, Riverton, Wyoming",42.994796,-108.371483,"90 Gas Hills Rd, Riverton, WY 82501, USA"


In [6]:
# save city location data to city_data_inc_latlog.csv
location_data_sunday.to_csv('Source_Data//city_data_inc_latlog.csv', index=False)

In [ ]:
pollution_summary = pollution_summary.reset_index()
pollution_summary = pollution_summary[pollution_summary['Unique Dates'] >= 350]
pollution_summary = pollution_summary.groupby(['City', 'County', 'State']).agg({'year': ['min', 'max']})
pollution_summary.columns = ['Min Year', 'Max Year']
pollution_summary['Year Difference'] = pollution_summary['Max Year'] - pollution_summary['Min Year']
pollution_summary = pollution_summary.sort_values('Year Difference', ascending=False)
print(pollution_summary)
pollution_data_available = pollution_summary.reset_index()
pollution_data_available

In [ ]:
stations_fixed = pd.read_fwf('source_data/ghcnd-stations.txt', widths=[11, 9, 10, 7, 3, 31, 4, 4, 5], header=None)
stations_fixed.columns = ['StationId', 'Latitude', 'Longitude', 'Elevation', 'StateCode', 'Name', 'GSN Flag', 'HCN/CRN Flag', 'WMO ID']
stations_fixed['Country'] = stations_fixed['StationId'].str[:2]
stations_fixed = stations_fixed[stations_fixed['Country'] == 'US']
states_fixed = pd.read_fwf('source_data/ghcnd-states.txt', widths=[2, 46], header=None)
states_fixed.columns = ['StateCode', 'State']
print("states_fixed", states_fixed.head())
print("stations_fixed", stations_fixed.head())
stations_fixed = pd.merge(stations_fixed, states_fixed, on='StateCode')
print("stations_fixed_post_merge",stations_fixed.head())

In [ ]:
print(pollution_summary.columns)
print(pollution_summary.head())
print(states_fixed.columns)
print(states_fixed.head())
print(stations_fixed.columns)
print(stations_fixed.head())

In [ ]:
pollution_summary = pollution_summary.reset_index()
pollution_summary['State'] = pollution_summary['State'].str.upper().str.strip()
states_fixed['State'] = states_fixed['State'].str.upper().str.strip()
pollution_summary_inc_sc = pd.merge(pollution_summary, states_fixed, left_on='State', right_on='State')
print("pollution_summary_post_merge", pollution_summary_inc_sc.head())
stations_fixed_gb = stations_fixed.groupby(['StateCode', 'State'])['StationId'].count().reset_index()
stations_fixed_gb.columns = ['StateCode', 'State', 'Station Count']
print("stations_fixed_station_count", stations_fixed_gb.head())
pollution_summary_stat_count = pd.merge(pollution_summary_inc_sc, stations_fixed_gb, on='StateCode')
print("pollution_summary_ merge2", pollution_summary_stat_count.head())
states_fixed = pd.merge(states_fixed, stations_fixed_gb, on='State')
print("states_fixed_stations_merge", states_fixed.head())
states_fixed = states_fixed.sort_values('Station Count', ascending=False)
print("states_fixed_sort", states_fixed)
pollution_summary_stat_count

In [ ]:
states_10_years = pollution_summary_stat_count[pollution_summary_stat_count['Year Difference'] >= 10]

In [ ]:
station_list = states_10_years['StationId'].tolist()
station_list